# pandas的拼接操作

In [4]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

### 回顾numpy的级联

In [5]:
nd1 = np.array([1,2,3])
nd2 = np.array([4,5,6,7])
np.concatenate([nd1,nd2])

array([1, 2, 3, 4, 5, 6, 7])

为方便理解，我们首先定义一个生成DataFrame的函数：

In [6]:
def make_df(cols,inds):
    data  = {c:[c+str(i) for i in inds] for c in cols}
    return DataFrame(data, index=inds)

In [7]:
df1 = make_df(list("abc"),[1,2,3])
df1

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [8]:
df2 = make_df(list('abc'),[4,5,6])
df2

,a,b,c
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


### 1. 使用pd.concat()级联
pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：

pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
#### 1) 简单级联
和np.concatenate一样，优先增加行数（默认axis=0）

In [9]:
pd.concat([df1,df2])

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


In [10]:
df3 = make_df(list('def'),[1,2,3])
df3

,d,e,f
1,d1,e1,f1
2,d2,e2,f2
3,d3,e3,f3


In [11]:
pd.concat([df1,df3], sort=True)  # 加上sort=True可以不报红

,a,b,c,d,e,f
1,a1,b1,c1,NaN,NaN,NaN
2,a2,b2,c2,NaN,NaN,NaN
3,a3,b3,c3,NaN,NaN,NaN
1,NaN,NaN,NaN,d1,e1,f1
2,NaN,NaN,NaN,d2,e2,f2
3,NaN,NaN,NaN,d3,e3,f3


In [12]:
pd.concat([df1,df3],axis=1)

,a,b,c,d,e,f
1,a1,b1,c1,d1,e1,f1
2,a2,b2,c2,d2,e2,f2
3,a3,b3,c3,d3,e3,f3


In [13]:
"""
总结：
    可以通过设置axis来改变级联方向
    这个axis的使用和np.concatenate一样的， 当axis = 0 的时候增加的行数， 如果不想有空值的出现，列应该是相同的
    当axis = 1的时候， 增加的是列数， 如果不想有空值的出现， 行索引应该是一样的
"""

'\n总结：\n    可以通过设置axis来改变级联方向\n    这个axis的使用和np.concatenate一样的， 当axis = 0 的时候增加的行数， 如果不想有空值的出现，列应该是相同的\n    当axis = 1的时候， 增加的是列数， 如果不想有空值的出现， 行索引应该是一样的\n'

In [14]:
df4 = make_df(list('abc'),inds=[2,3,4])
df4

,a,b,c
2,a2,b2,c2
3,a3,b3,c3
4,a4,b4,c4


In [15]:
pd.concat([df1,df4])

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3
2,a2,b2,c2
3,a3,b3,c3
4,a4,b4,c4


也可以选择忽略ignore_index，重新索引

In [16]:
pd.concat([df1,df4],ignore_index=True)

,a,b,c
0,a1,b1,c1
1,a2,b2,c2
2,a3,b3,c3
3,a2,b2,c2
4,a3,b3,c3
5,a4,b4,c4


或者使用多层索引 keys

concat([x,y],keys=['x','y'])

In [17]:
# 设置多层索引
pd.concat([df1,df4], keys = ["一班","二班"])

a   b   c
一班 1  a1  b1  c1
   2  a2  b2  c2
   3  a3  b3  c3
二班 2  a2  b2  c2
   3  a3  b3  c3
   4  a4  b4  c4

#### 2) 不匹配级联
不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

In [18]:
df1

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [19]:
df5 = make_df(list('abcd'),[3,4,5,6])
df5

,a,b,c,d
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


In [20]:
pd.concat([df1,df5],sort=True)

,a,b,c,d
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


有3种连接方式：

    外连接：补NaN（默认模式）

In [21]:
pd.concat([df1,df5],join="outer",sort=True)

,a,b,c,d
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


    内连接：只连接匹配的项

In [22]:
pd.concat([df1,df5],join="inner")  #inner 内连接， 只连接匹配的项

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3
3,a3,b3,c3
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


    连接指定轴 join_axes

In [23]:
df6 = make_df(list("abcd"),[3,4,5,6])
df6

,a,b,c,d
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


In [24]:
df6.columns

Index(['a', 'b', 'c', 'd'], dtype='object')

In [25]:
pd.concat([df6,df5,df2,df1],axis=1)  # axis 轴  axes 面

,a,b,c,d,a,b,c,d,a,b,c,a,b,c
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a1,b1,c1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a2,b2,c2
3,a3,b3,c3,d3,a3,b3,c3,d3,NaN,NaN,NaN,a3,b3,c3
4,a4,b4,c4,d4,a4,b4,c4,d4,a4,b4,c4,NaN,NaN,NaN
5,a5,b5,c5,d5,a5,b5,c5,d5,a5,b5,c5,NaN,NaN,NaN
6,a6,b6,c6,d6,a6,b6,c6,d6,a6,b6,c6,NaN,NaN,NaN


#### 3) 使用append()函数添加
由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [26]:
s1 = ["123"]
s1.append("345")

In [27]:
s1

['123', '345']

In [28]:
df1

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [29]:
df2

,a,b,c
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


In [30]:
df1.append(df2) #追加  他的功能比concat少很多，所以在用的时候推荐使用concat

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


### 2. 使用pd.merge()合并
两个DataFrame一般要有两个相同的列名字， 才可以进行合并， 融合

merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

#### 1）一对一合并

In [36]:
df1 = DataFrame({"age":[30,22,36],
                "work":["teach","accounting","sell"],
                "sex":["man","woman","woman"]})
df1

,age,work,sex
0,30,teach,man
1,22,accounting,woman
2,36,sell,woman


In [37]:
df2 = DataFrame({"home":["Beijing","Shanghai","Guangzhou"],
                "work":["teach","accounting","sell"],
                "weight":[60,55,54]})
df2

,home,work,weight
0,Beijing,teach,60
1,Shanghai,accounting,55
2,Guangzhou,sell,54


In [38]:
pd.concat([df1,df2],axis=1)

,age,work,sex,home,work,weight
0,30,teach,man,Beijing,teach,60
1,22,accounting,woman,Shanghai,accounting,55
2,36,sell,woman,Guangzhou,sell,54


In [39]:
df1.merge(df2)

,age,work,sex,home,weight
0,30,teach,man,Beijing,60
1,22,accounting,woman,Shanghai,55
2,36,sell,woman,Guangzhou,54


#### 2) 多对一合并

In [40]:
df1

,age,work,sex
0,30,teach,man
1,22,accounting,woman
2,36,sell,woman


In [41]:
df3 = DataFrame({"home":["Shenzhen","Beijing","Shanghai","Anhui","Shandong"],
                "work":["teach","teach","teach","accounting","sell"],
                "weight":[60,75,80,50,40]})
df3

,home,work,weight
0,Shenzhen,teach,60
1,Beijing,teach,75
2,Shanghai,teach,80
3,Anhui,accounting,50
4,Shandong,sell,40


In [42]:
df1.merge(df3)

,age,work,sex,home,weight
0,30,teach,man,Shenzhen,60
1,30,teach,man,Beijing,75
2,30,teach,man,Shanghai,80
3,22,accounting,woman,Anhui,50
4,36,sell,woman,Shandong,40


#### 多对多合并

In [43]:
df5  = DataFrame({'age':[28,30,22,36],
                "work":["teach","teach", "accounting","sell"],
                 "sex":["women","man","woman","woman"]})
df5

,age,work,sex
0,28,teach,women
1,30,teach,man
2,22,accounting,woman
3,36,sell,woman


In [44]:
df3

,home,work,weight
0,Shenzhen,teach,60
1,Beijing,teach,75
2,Shanghai,teach,80
3,Anhui,accounting,50
4,Shandong,sell,40


In [45]:
df3.merge(df5)

,home,work,weight,age,sex
0,Shenzhen,teach,60,28,women
1,Shenzhen,teach,60,30,man
2,Beijing,teach,75,28,women
3,Beijing,teach,75,30,man
4,Shanghai,teach,80,28,women
5,Shanghai,teach,80,30,man
6,Anhui,accounting,50,22,woman
7,Shandong,sell,40,36,woman


#### key的规范化
使用on=显式指定哪一列为key,当有多个key相同时使用

In [47]:
df5

,age,work,sex
0,28,teach,women
1,30,teach,man
2,22,accounting,woman
3,36,sell,woman


In [48]:
df6 = DataFrame({"age":[30,22,37],
                "work":["teach","leader","sell"],
                "hobby":['sixdog',"playcat","diaofish"]})
df6

,age,work,hobby
0,30,teach,sixdog
1,22,leader,playcat
2,37,sell,diaofish


In [49]:
df5.merge(df6,on="age")  # 使用suffixes=["_xxx","_xxx"] 可指定后缀，替代_x,_y

,age,work_x,sex,work_y,hobby
0,30,teach,man,teach,sixdog
1,22,accounting,woman,leader,playcat


使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用

In [50]:
df5

,age,work,sex
0,28,teach,women
1,30,teach,man
2,22,accounting,woman
3,36,sell,woman


In [51]:
df7 = DataFrame({"体重":[30,22,36],
                "工作":["teach","accounting","sell"],
                "爱好":["man","woman","woman"]})
df7

,体重,工作,爱好
0,30,teach,man
1,22,accounting,woman
2,36,sell,woman


In [54]:
df5.merge(df7,left_on="work",right_on="工作").drop("work",axis=1)

,age,sex,体重,工作,爱好
0,28,women,30,teach,man
1,30,man,30,teach,man
2,22,woman,22,accounting,woman
3,36,woman,36,sell,woman


In [55]:
df5

,age,work,sex
0,28,teach,women
1,30,teach,man
2,22,accounting,woman
3,36,sell,woman


In [58]:
s = df5[["age"]] * 1000
s.columns = ["salary"]
s

,salary
0,28000
1,30000
2,22000
3,36000


In [60]:
df5.merge(s, left_index=True,right_index=True)

,age,work,sex,salary
0,28,teach,women,28000
1,30,teach,man,30000
2,22,accounting,woman,22000
3,36,sell,woman,36000


In [61]:
pd.concat([df5,s],axis=1)  # 效果同上

,age,work,sex,salary
0,28,teach,women,28000
1,30,teach,man,30000
2,22,accounting,woman,22000
3,36,sell,woman,36000


#### 5) 内合并与外合并
    内合并：只保留两者都有的key（默认模式）

In [62]:
df3

,home,work,weight
0,Shenzhen,teach,60
1,Beijing,teach,75
2,Shanghai,teach,80
3,Anhui,accounting,50
4,Shandong,sell,40


In [63]:
df6

,age,work,hobby
0,30,teach,sixdog
1,22,leader,playcat
2,37,sell,diaofish


In [64]:
df3.merge(df6,how="inner")

,home,work,weight,age,hobby
0,Shenzhen,teach,60,30,sixdog
1,Beijing,teach,75,30,sixdog
2,Shanghai,teach,80,30,sixdog
3,Shandong,sell,40,37,diaofish


    外合并 how='outer'：补NaN

In [65]:
df3.merge(df6,how="outer")

,home,work,weight,age,hobby
0,Shenzhen,teach,60.0,30.0,sixdog
1,Beijing,teach,75.0,30.0,sixdog
2,Shanghai,teach,80.0,30.0,sixdog
3,Anhui,accounting,50.0,NaN,NaN
4,Shandong,sell,40.0,37.0,diaofish
5,NaN,leader,NaN,22.0,playcat


    左合并、右合并：how='left'，how='right'

In [66]:
df3.merge(df6,how="left")

,home,work,weight,age,hobby
0,Shenzhen,teach,60,30.0,sixdog
1,Beijing,teach,75,30.0,sixdog
2,Shanghai,teach,80,30.0,sixdog
3,Anhui,accounting,50,NaN,NaN
4,Shandong,sell,40,37.0,diaofish


In [67]:
df3.merge(df6,how="right")

,home,work,weight,age,hobby
0,Shenzhen,teach,60.0,30,sixdog
1,Beijing,teach,75.0,30,sixdog
2,Shanghai,teach,80.0,30,sixdog
3,Shandong,sell,40.0,37,diaofish
4,NaN,leader,NaN,22,playcat


#### 6) 列冲突的解决
当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

In [ ]:
"""
总结：merge() 参数：on    keys    left_on,right_on    left_index,right_index    how  suffixes
"""